## Polars vs Dask

- **Load**: CSV, Excel, Parquet, SQLite (via pandas interop where needed)
- **Transform**: filter, select, derive columns, groupby/aggregate
- **Compute model**: Polars eager vs lazy; Dask lazy with `.compute()`

We'll use an included `temperatures.csv` and generate equivalent Excel/Parquet/SQLite files for demos.


In [ ]:
%pip install polars
%pip install dask[dataframe]
%pip install pandas
%pip install pyarrow
%pip install openpyxl
%pip install sqlite-utils
%pip install fastexcel


In [ ]:
# grab the data if not exists

import urllib.request
import os

csv_path = "temperatures.csv"
excel_path = "temperatures.xlsx"
parquet_path = "temperatures.parquet"
sqlite_path = "temperatures.sqlite"

if not os.path.exists(csv_path):
    url = 'https://raw.githubusercontent.com/DSE200/Day1/main/temperatures.csv'
    try:
        urllib.request.urlretrieve(url, "temperatures.csv")
    except Exception as e:
        raise RuntimeError(f"Failed to download temperatures.csv from {url}: {e}")

if not os.path.exists(excel_path):
    url = 'https://raw.githubusercontent.com/DSE200/Day1/main/temperatures.xlsx'
    try:
        urllib.request.urlretrieve(url, "temperatures.xlsx")
    except Exception as e:
        raise RuntimeError(f"Failed to download temperatures.xlsx from {url}: {e}")

if not os.path.exists(parquet_path):
    url = 'https://raw.githubusercontent.com/DSE200/Day1/main/temperatures.parquet'
    try:
        urllib.request.urlretrieve(url, "temperatures.parquet")
    except Exception as e:
        raise RuntimeError(f"Failed to download temperatures.parquet from {url}: {e}")

if not os.path.exists(sqlite_path):
    url = 'https://raw.githubusercontent.com/DSE200/Day1/main/temperatures.sqlite'
    try:
        urllib.request.urlretrieve(url, "temperatures.sqlite")
    except Exception as e:
        raise RuntimeError(f"Failed to download temperatures.sqlite from {url}: {e}")


In [ ]:
# Loading data with Pandas
import pandas as pd
import sqlite3


table_name = "temperatures"

pd_csv = pd.read_csv(csv_path)
pd_parquet = pd.read_parquet(parquet_path)
pd_excel = pd.read_excel(excel_path)
# SQLite
with sqlite3.connect(sqlite_path) as conn:
    pd_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)


In [ ]:
# Simple transformations with Pandas

pd_res = (
    pd_csv.dropna(subset=["city", "temperature (C)"]) # drop the rows with null city and temperature (c)
         .rename(columns={"temperature (C)": "temp_C"}) # rename the temperature (c) to temp_C
)
pd_res["temp_F"] = pd_res["temp_C"] * 9/5 + 32 # create a new column temp_F



In [ ]:
# Group by city and compute mean temp in C/F
pd_res = (pd_res.groupby("city")[["temp_C", "temp_F"]].mean()
                 .rename(columns={"temp_C": "mean_C", "temp_F": "mean_F"})
                 .reset_index()
                 .sort_values("mean_C", ascending=False))
pd_res.head(10)

In [ ]:
# Introduction to Polars and dask
import polars as pl



# Polars: eager reads
pl_csv = pl.read_csv(csv_path)
pl_parquet = pl.read_parquet(parquet_path)
pl_excel = pl.read_excel(excel_path)

# Polars: SQLite via pandas interop
with sqlite3.connect(sqlite_path) as conn:
    pd_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)
pl_sql = pl.from_pandas(pd_sql)



In [ ]:


import dask.dataframe as dd

# Dask: lazy reads (need .compute())
# CSV
dd_csv = dd.read_csv(csv_path) # nothing is actually read yet until we call compute

# Dask read parquet
dd_parquet = dd.read_parquet(parquet_path)

# Excel via pandas + from_pandas
pd_xl = pd.read_excel(excel_path)
dd_excel = dd.from_pandas(pd_xl, npartitions=1)

# SQLite via pandas + from_pandas
with sqlite3.connect(sqlite_path) as conn:
    pd_sql = pd.read_sql(f"SELECT * FROM {table_name}", conn)
dd_sql = dd.from_pandas(pd_sql, npartitions=1)



### Comparing
Goal: select `date`, `city`, keep temp > 25°C, top 5 by temp.


In [ ]:
# pandas
mini_pd = (pd_csv
    .rename(columns={"temperature (C)": "temp_C"})
    .loc[lambda d: d["temp_C"] > 25, ["date", "city", "temp_C"]]
    .sort_values("temp_C", ascending=False)
    .head(5).reset_index(drop=True)
)
mini_pd


In [ ]:
# Polars
mini_pl = (pl_csv
    .with_columns(pl.col("temperature (C)").alias("temp_C"))
    .filter(pl.col("temp_C") > 25)
    .select(["date", "city", "temp_C"])
    .sort("temp_C", descending=True)
    .head(5)
)
mini_pl


In [ ]:
# Dask
mini_dd = (dd_csv
    .rename(columns={"temperature (C)": "temp_C"})
    .loc[lambda d: d["temp_C"] > 25, ["date", "city", "temp_C"]]
    .nlargest(5, "temp_C").reset_index(drop=True)
)
mini_dd.compute()


### Side-by-side: filter, select, derive, groupby
We'll demonstrate identical logic:
- Keep rows with non-null `city` and `temperature (C)`
- Derive `temp_F = temperature (C) * 9/5 + 32`
- Group by `city`, compute mean temp in C/F
- Sort by mean temp desc


In [ ]:
# Polars (eager)
res_pl = (
    pl_csv
    .filter(pl.col("city").is_not_null() & pl.col("temperature (C)").is_not_null())
    .with_columns(pl.col("temperature (C)").alias("temp_C"))
    .with_columns((pl.col("temp_C") * 9/5 + 32).alias("temp_F"))
    .group_by("city")
    .agg([pl.col("temp_C").mean().alias("mean_C"), pl.col("temp_F").mean().alias("mean_F")])
    .sort(pl.col("mean_C"), descending=True)
)
res_pl.head(10)


### Lazy vs Eager at a glance
- **Polars eager**: `pl.read_csv(...).filter(...).group_by(...).collect()` is immediate unless using lazy.
- **Polars lazy**: use `pl.scan_csv(...)` to build a plan, `collect()` to execute.
- **Dask**: always lazy; chain ops, then `.compute()` to execute.


In [ ]:
# Polars LAZY pipeline
dsl = (
    pl.scan_csv(str(csv_path))
    .filter(pl.col("city").is_not_null() & pl.col("temperature (C)").is_not_null())
    .with_columns(pl.col("temperature (C)").alias("temp_C"))
    .with_columns((pl.col("temp_C") * 9/5 + 32).alias("temp_F"))
    .group_by("city")
    .agg([pl.col("temp_C").mean().alias("mean_C"), pl.col("temp_F").mean().alias("mean_F")])
    .sort(pl.col("mean_C"), descending=True)
)
# Explain physical plan (optional)
print("Polars lazy plan:")
print(dsl.explain())
res_pl_lazy = dsl.collect().head(5)
res_pl_lazy


1. Grab and scan the CSV file and grab 2 out of 3 columns (Temperature and city)
2. Select only the non null rows
3. Rename the column temperature c
4. Create column for temp_F
5. Group By city
5. Sort


In [ ]:
# Dask lazy pipeline (always lazy)
pipe = (
    dd.read_csv(str(csv_path))
    .dropna(subset=["city", "temperature (C)"])
    .rename(columns={"temperature (C)": "temp_C"})
    .assign(temp_F=lambda x: x["temp_C"] * 9/5 + 32)
    .groupby("city")["temp_C", "temp_F"].mean().reset_index()
    .rename(columns={"temp_C": "mean_C", "temp_F": "mean_F"})
)
print(pipe)
res_dask_lazy = pipe.compute().sort_values("mean_C", ascending=False).head(5)
res_dask_lazy


Polars Lazy vs Dask
1. Polars operate in RAM vs Dask runs through the partitions to support larger than memory datasets
2. Polars is single threaded vs Multi threaded dask
3. Polars has advanced query optimization similar to database system. Dask's priority is scallability and out of core datasets.